In [1]:
import json
import torch
import traceback
import itertools

In [52]:
__model__ = None

__feature_neams__ = ['TotalSteps', 'TotalDistance', 'TrackerDistance', 'LoggedActivitiesDistance', 'VeryActiveDistance', 'ModeratelyActiveDistance', 'LightActiveDistance', 'SedentaryActiveDistance', 'VeryActiveMinutes', 'FairlyActiveMinutes', 'LightlyActiveMinutes', 'SedentaryMinutes', 'Calories', 'StepTotal']

__spot_feature__ = None
__spot_names__ = None

In [59]:
request = '{"contexts":[[-0.7979, -0.8019, -0.8019,  0.0000, -0.5425, -0.3981, -0.4444,  0.0000, -0.5429, -0.4084, -0.2121, -0.2240, -0.3177, -0.7979]]}'
request = json.loads(request)
contexts = request["contexts"]
feature_num = request.get('feature_num', 5)
spot_num = request.get('spot_num', 3)


assert isinstance(contexts, list), 'Expect contexts to be a list'
assert all(isinstance(ctx, list) and len(ctx) == 14 for ctx in contexts), 'Expect all list in contexts has 14 features'

global __model__, __spot_feature__, __spot_names__, __feature_neams__
if __model__ is None:
    __model__ = torch.load('daily_model.pth')
if __spot_feature__ is None:
    with open('spot.json') as f:
        spot = json.load(f)
    __spot_feature__ = torch.tensor(list(spot.values())).float()
    __spot_names__ = list(spot.keys())

results = []
for context in contexts:
    x = torch.tensor(context, requires_grad=True)[None, :]
    y = __model__['pred_model'](__model__['auto_encoder'](x))

    g, = torch.autograd.grad(y, x)  # dy/dx
    g.squeeze_()

    s = __spot_feature__ @ g

    results.append({
        'features': [{'feature': __feature_neams__[i], 'increase:':bool(g[i]>0), 'sorce':abs(g[i].item()), 'grad':g[i].item()} for i in itertools.islice(torch.argsort(g, descending=True), feature_num)],
        'spots': [{'spot': __spot_names__[i], 'sorce': s[i].item()} for i in itertools.islice(torch.argsort(s, descending=True), spot_num)]
    })

json.dumps({"return": results})

'{"return": [{"features": [{"feature": "SedentaryMinutes", "increase:": true, "sorce": 0.02699221856892109, "grad": 0.02699221856892109}, {"feature": "TotalDistance", "increase:": true, "sorce": 0.015108737163245678, "grad": 0.015108737163245678}, {"feature": "FairlyActiveMinutes", "increase:": true, "sorce": 0.012789390981197357, "grad": 0.012789390981197357}, {"feature": "LightlyActiveMinutes", "increase:": true, "sorce": 0.008207416161894798, "grad": 0.008207416161894798}, {"feature": "TotalSteps", "increase:": true, "sorce": 0.007286544423550367, "grad": 0.007286544423550367}], "spots": [{"spot": "walking brisk", "sorce": 0.46626126766204834}, {"spot": "bicycling", "sorce": 0.427340030670166}, {"spot": "running", "sorce": 0.3752664029598236}]}]}'

In [ ]:
import torch

'''

TotalSteps
TotalDistance
TrackerDistance
LoggedActivitiesDistance
VeryActiveDistance
ModeratelyActiveDistance
LightActiveDistance
SedentaryActiveDistance
VeryActiveMinutes
FairlyActiveMinutes
LightlyActiveMinutes
SedentaryMinutes
Calories
StepTotal

'''

def get_recommendation(x):
    saved_model = torch.load('daily_model.pth')
    x = torch.tensor(x, requires_grad=True)[None, :]
    y = saved_model['pred_model'](saved_model['auto_encoder'](x))

    g, = torch.autograd.grad(y, x) # dy/dx
    
    return g[0].tolist()

get_recommendation([-0.7979, -0.8019, -0.8019,  0.0000, -0.5425, -0.3981, -0.4444,  0.0000, -0.5429, -0.4084, -0.2121, -0.2240, -0.3177, -0.7979])

$$
input = \frac{x-\mu}{\sigma}
$$

$$
\mu_{i+1} = (1-\alpha)\cdot\mu_{i} + \alpha\cdot x
$$
$$
\chi_{i+1} = (1-\alpha)\cdot\chi_{i} + \alpha\cdot x^2
$$
$$
\sigma_{i+1} = \sqrt{\chi_{i+1}-\mu^2_{i+1}}
$$

In [ ]:
mean, chi = 0, 0
x = 1
def sqrt(x):
    return 0
def get_data():
    return 0

a = 0.1
mean = None
chi  = None
while True:
    x = get_data()

    mean = (1 - a) * mean + a * x    if mean is not None else x
    chi  = (1 - a) * chi  + a * x**2 if chi  is not None else x**2

    std = sqrt(chi - mean**2)
    y = (x - mean) / (std if std > 0 else 1)

In [ ]:
import numpy as np

class running_z_sorce:
    def __init__(self, a:float=0.1) -> None:
        self.mean = None
        self.chi  = None
        self.a = a
    
    def get(self, xs):
        self.mean = [(1 - self.a) * m + self.a * x    for x, m in zip(xs, self.mean)] if self.mean is not None else [x    for x in xs]
        self.chi  = [(1 - self.a) * c + self.a * x**2 for x, c in zip(xs, self.chi )] if self.chi  is not None else [x**2 for x in xs]

        std = [np.sqrt(c - m**2) for m, c in zip(self.mean, self.chi)]
        return [(x - m) / (s if s > 0 else 1) for x, m, s in zip(xs, self.mean, std)]

instance = running_z_sorce()
print(instance.get([1, 2, 3]))
print(instance.get([2, 0, 0]))
print(instance.get([1, 2, 2]))
print(instance.get([2, 3, 1]))